# Specific query methods

#### Sections
* [`session_prices`](#session_prices)
* [`close_at`](#close_at)
* [`price_at`](#price_at)
* [`price_range`](#price_range)

#### Note

The cell **outputs** shown in this tutorial are based on executing the cells at the time shown in the output of the following cell. Simply rerun the cells to bring any dynamic output up to date.

In [2]:
import pandas as pd
from zoneinfo import ZoneInfo
now = pd.Timestamp.now(tz=ZoneInfo("UTC")).floor("T")
print(f"{now!r}")
print(f"{now.astimezone(ZoneInfo('America/New_York'))!r}")

Timestamp('2022-05-13 13:26:00+0000', tz='UTC')
Timestamp('2022-05-13 09:26:00-0400', tz='America/New_York')


## Setup

Run the following cell to import tutorial dependencies.

In [3]:
import pandas as pd
from market_prices import PricesYahoo
from market_prices.support import tutorial_helpers as th

Run the following cell to define values used in this tutorial.

In [4]:
_prices_mix = PricesYahoo("MSFT, 9988.HK, AZN.L")
xnys = _prices_mix.calendars["MSFT"]
xhkg = _prices_mix.calendars["9988.HK"]
xlon = _prices_mix.calendars["AZN.L"]
_calendars = [xnys, xhkg, xlon]
_session_length = [
    pd.Timedelta(hours=6, minutes=30),
    pd.Timedelta(hours=6, minutes=30),
    pd.Timedelta(hours=8, minutes=30),
]
# get session for which price data available at all base intervals
_sessions_range = th.get_sessions_range_for_bi(
    _prices_mix, _prices_mix.bis.T1
)
session = th.get_conforming_sessions(
    _calendars, _session_length, *_sessions_range, 1
)[0]

# get session for which intraday price data not available < T5
_sessions_range = th.get_sessions_range_for_bi(
    _prices_mix, _prices_mix.bis.T5
)
sessionT5 = th.get_conforming_sessions(
    _calendars, _session_length, *_sessions_range, 2
)[-1]

## Specific query methods

The Prices class offers some methods to query prices for a specific session, time or period. They all return a single-row `DataFrame`.

Execute the following cell to define a Prices instance that will be used to demonstrate these methods. The symbols correspond with the following equities:
* Microsoft, New York Stock Exchange listing.
* AstraZeneca, London Stock Exchange listing.
* Alibaba, Hong Kong Stock Exchange listing.

Note that the `lead_symbol` is AstraZeneca.

In [5]:
prices = PricesYahoo("MSFT, AZN.L, 9988.HK", lead_symbol="AZN.L")

## `session_prices`
`session_prices` is the quickest way to get prices for a specific session. 

The method can take a single argument that represents an actual session for at least one of the symbols. The argument can take a `Timestamp` or any other type that can be passed as a single argument to `Timestamp`.

In [6]:
prices.session_prices("2022-04-26")

symbol       MSFT                                             AZN.L           \
             open        high    low       close    volume     open     high   
2022-04-26  277.5  278.359985  270.0  270.220001  46518400  10338.0  10436.0   

symbol                                9988.HK                               \
                low    close   volume    open       high    low      close   
2022-04-26  10218.0  10332.0  3205465    84.5  87.900002  83.75  84.900002   

symbol                
              volume  
2022-04-26  38304821

A `ValueError` will be raised if the passed value does not represent a session for any of the symbols.

In [ ]:
prices.session_prices("2022-04-24")  # a sunday

```
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-12-cddffb6db884> in <module>
----> 1 prices.session_prices("2022-04-24")  # a sunday

ValueError: 2022-04-24 00:00:00 is not a session of any associated calendar.
```

`session_prices` can be called without an argument to return prices for the most recent session of any symbol.

In [8]:
prices.session_prices()

symbol     MSFT                          AZN.L                            \
           open high low close volume     open     high     low    close   
2022-05-13  NaN  NaN NaN   NaN    NaN  10054.0  10300.0  9996.0  10250.0   

symbol                9988.HK                                         
            volume       open       high    low      close    volume  
2022-05-13  682594  82.199997  82.849998  81.25  82.199997  53688518

Pass `stack` as `True` to stack the symbols to separate rows.

In [9]:
prices.session_prices(stack=True)

open          high      low         close  \
           symbol                                                       
2022-05-13 9988.HK     82.199997     82.849998    81.25     82.199997   
           AZN.L    10054.000000  10300.000000  9996.00  10250.000000   
           MSFT              NaN           NaN      NaN           NaN   

                        volume  
           symbol               
2022-05-13 9988.HK  53688518.0  
           AZN.L      682594.0  
           MSFT            NaN

## `close_at`

`close_at` returns the prices as at the end of specific 'day'. Like `session_prices` it takes a single argument although differs in that the passed date can be a date when one, some or none of the associated calendars were open. If a symbol does not trade on the queried date then the price will be as at its prior close. This can be seen by considering the following price table against the subsequent call to `close_at`.

In [10]:
prices.get("1D", "2021-12-23", "2021-12-29", close_only=True)

symbol,MSFT,AZN.L,9988.HK
2021-12-23,334.690002,8596.0,113.300003
2021-12-24,NaN,8611.0,113.000000
2021-12-27,342.450012,NaN,NaN
2021-12-28,341.250000,NaN,113.500000
2021-12-29,341.950012,8656.0,110.599998


In [11]:
prices.close_at("2021-12-28")

symbol,MSFT,AZN.L,9988.HK
2021-12-28,341.25,8611.0,113.5


The price for AZN.L can be seen to have been carried forward from the close on the 24th.

The method can also be called with no argument, in which the price for each symbol will reflect:
* the most recent close if the symbol's exchange is currently closed.
* the most recent price if the symbol's exchange is currently open.

In [12]:
prices.close_at()

symbol,MSFT,AZN.L,9988.HK
2022-05-13,255.350006,10250.0,82.199997


## `price_at`

`price_at` returns the most recent prices available as at a specific minute.

When no arguments are passed the method returns the most recent prices as of 'now'.

In [13]:
prices.price_at()

symbol,MSFT,AZN.L,9988.HK
2022-05-13 14:26:00+01:00,255.350006,10250.0,82.199997


Alternatively a specific minute can be specified.

In [14]:
minute = xlon.session_close(session) - pd.Timedelta(47, "T")
minute_ = minute.astimezone(prices.tz_default)
print(f"{minute_=}\n")  # for reference

prices.price_at(minute)

minute_=Timestamp('2022-04-14 15:43:00+0100', tz='Europe/London')



symbol,MSFT,AZN.L,9988.HK
2022-04-14 15:43:00+01:00,283.570007,10526.0,95.5


If a symbol is not open at the minute, the price will reflect its most recent close price. At the passed minute in the above example only the London and New York markets were open. The price for Alibaba is therefore its close price from earlier in the day. **Although not all symbols may be open at the minute, a symbol's price will always reflect the most recently _available_ 'price at' that time**.

By default the indice has the same timezone as the Price instance's default timezone, in this case London. The `tz` parameter can be passed to override the default timezone. It can take the same types as the `get` method's 'tzout' parameter and works in the same way.

In [15]:
prices.price_at(minute, tz="America/New_York")

symbol,MSFT,AZN.L,9988.HK
2022-04-14 10:43:00-04:00,283.570007,10526.0,95.5


In [16]:
prices.price_at(minute, tz="9988.HK")

symbol,MSFT,AZN.L,9988.HK
2022-04-14 22:43:00+08:00,283.570007,10526.0,95.5


Note: The `tz` parameter also defines the timezone of `minute` in the event that `minute` is otherwise timezone-naive. In this way the `tz` parameter can be thought of as combining the `get` method's parameters 'tzin' and 'tzout'.

Passing a minute as a date will raise an error. To pass a minute as `midnight` `minute` must have a timezone, either by passing `tz` or passing `minute` as a timezone-aware timestamp.

In [17]:
midnight = minute.ceil("D")
print(f"{midnight=}\n")  # for reference

prices.price_at(midnight)

midnight=Timestamp('2022-04-15 00:00:00+0000', tz='UTC')



symbol,MSFT,AZN.L,9988.HK
2022-04-14 21:00:00+01:00,279.779999,10522.0,95.5


Notice that although `minute` was passed as midnight, the indice reflects the New York Stock Exchange close (indice timezone is London). The **indice is always defined as `minute` or the most recent minute prior to `minute` EITHER at which data is available for any symbol and at least one of the underlying exchanges was open OR an underlying exchange closed**...

In the above example no exchange was open at UTC midnight, hence the indice is the most recent close, in this case the New York close.

Consider the following example for which underlying data is not available at any base interval smaller than 5 minutes (see the [data_availability](./data_availability.ipynb) tutorial for an explanation of base intervals).

In [18]:
minute = xlon.session_close(sessionT5) - pd.Timedelta(1, "T")
minute_ = minute.astimezone(prices.tz_default)
print(f"{minute_=}\n")  # for reference

prices.price_at(minute)

minute_=Timestamp('2022-03-16 16:29:00+0000', tz='Europe/London')



symbol,MSFT,AZN.L,9988.HK
2022-03-16 16:25:00+00:00,290.51001,9415.0,90.699997


Although `minute` is passed as 16:29, one minute before the London close, the indice is 16:25. In this case, as the smallest base interval available to serve the request is T5, the most recent price prior to `minute` at which data is available is the open price of the indice that covers the period 16:25 through 16:30.

The effect of data availability is more pronounced when only daily data is available at the requested minute. The following cell shows the session prices for "2019-12-11", a session for which only daily data is available.

In [19]:
session_1D = "2019-12-11"
prices.session_prices(session_1D)

symbol            MSFT                                                  \
                  open        high         low       close      volume   
2019-12-11  151.539993  151.869995  150.330002  151.699997  18856600.0   

symbol       AZN.L                                        9988.HK              \
              open    high     low   close     volume        open        high   
2019-12-11  7290.0  7342.0  7273.0  7306.0  2270223.0  196.199997  198.399994   

symbol                                         
                   low       close     volume  
2019-12-11  195.399994  197.600006  6534411.0

Compare the above prices with the prices returned by `price_at` when requesting prices at one minute before the London close. 

In [20]:
one_min = pd.Timedelta(1, "T")
minute = xlon.session_close(session_1D) - one_min
print(f"{minute=}\n")  # for reference

prices.price_at(minute)

minute=Timestamp('2019-12-11 16:29:00+0000', tz='UTC')



symbol,MSFT,AZN.L,9988.HK
2019-12-11 14:30:00+00:00,151.539993,7290.0,197.600006


The only price data available to serve the request is daily, from which the only prices known as at a specific time are the session open and the session close. Therefore, at one minute prior to the London close the most recent known price for each symbol is:
* Alibaba - session close (Hong Kong closes 08:00 UTC)
* Microsoft - session open (New York opens prior to London's close)
* AstraZeneca - session open

The indice reflects the most recent minute, prior to `minute`, for which price data is available for any symbol, in this case the New York open. **NOTE**: the price for AstraZeneca at this time, a couple of hours or so before the London close, would NOT have been the price shown (the price shown is as at the London open). **When `price_at` is served from daily data the price given for each symbol is the most recent price available as at the indice, which may not reflect the actual price at this time** (this is not so when `price_at` is served from intraday price data, in which case prices will always be the 'price at' the indice).

Requesting prices at one minute later, i.e. the London close, shows how the price for AstraZeneca can now reflect the session close. The other prices remain the same although the indice moves forward to the London close.

In [21]:
prices.price_at(minute + one_min)

symbol,MSFT,AZN.L,9988.HK
2019-12-11 16:30:00+00:00,151.539993,7306.0,197.600006


## `price_range`

`price_range` returns OHLCV data for a period evaluated from period parameters.

In [22]:
prices.price_range(end=session, days=20)

symbol                                            MSFT              \
                                                  open        high   
(2022-03-18 01:30:00, 2022-04-14 21:00:00]  295.369995  315.950012   

symbol                                                              \
                                                   low       close   
(2022-03-18 01:30:00, 2022-04-14 21:00:00]  279.320007  279.829987   

symbol                                                    AZN.L           \
                                                 volume    open     high   
(2022-03-18 01:30:00, 2022-04-14 21:00:00]  579770300.0  9650.0  11000.0   

symbol                                                                   \
                                               low    close      volume   
(2022-03-18 01:30:00, 2022-04-14 21:00:00]  9482.0  10536.0  51707521.0   

symbol                                        9988.HK                         \
                                                 open        high        low   
(2022-03-18 01:30:00, 2022-04-14 21:00:00]  97.150002  119.400002  93.849998   

symbol                                                         
                                           close       volume  
(2022-03-18 01:30:00, 2022-04-14 21:00:00]  95.5  946826155.0

The following prices are offered for each symbol:
* 'open' - price at start of the period.
* 'high' - highest price registered during the period.
* 'low' - lowest price registered during the period.
* 'close' - price at end of the period.
* 'volume' - total volume registered over the period.

The indice expresses the period covered by the data.

Under-the-bonnet the method simply passes the parameters to the `get` method and then evaluates the aggregated data from the returned table. `price_range` can take any valid combination of period parameters that can be passed to `get` (see [periods](./periods.ipynb) tutorial).

Passing `underlying` as `True` returns the underlying table as the second item of a tuple.

In [23]:
minute = xnys.session_open(sessionT5) + pd.Timedelta(15, "T")
print(f"{minute=}\n")  # for reference

rng, df = prices.price_range(end=minute, years=3, underlying=True)
rng

minute=Timestamp('2022-03-16 13:45:00+0000', tz='UTC')



symbol                                            MSFT              \
                                                  open        high   
(2019-03-18 01:30:00, 2022-03-16 13:45:00]  116.169998  349.670013   

symbol                                                                        \
                                                   low   close        volume   
(2019-03-18 01:30:00, 2022-03-16 13:45:00]  115.519997  290.75  2.280511e+10   

symbol                                       AZN.L                           \
                                              open     high     low   close   
(2019-03-18 01:30:00, 2022-03-16 13:45:00]  6290.0  10120.0  5626.0  9452.0   

symbol                                                   9988.HK              \
                                                  volume    open        high   
(2019-03-18 01:30:00, 2022-03-16 13:45:00]  1.907237e+09   187.0  309.399994   

symbol                                                                     
                                             low      close        volume  
(2019-03-18 01:30:00, 2022-03-16 13:45:00]  71.0  90.699997  1.679227e+10

In [24]:
df

symbol                                            MSFT              \
                                                  open        high   
[2019-03-18, 2019-03-18)                    116.169998  117.610001   
[2019-03-19, 2019-03-19)                    118.089996  118.440002   
[2019-03-20, 2019-03-20)                    117.389999  118.750000   
[2019-03-21, 2019-03-21)                    117.139999  120.820000   
[2019-03-22, 2019-03-22)                    119.500000  119.589996   
...                                                ...         ...   
[2022-03-16 13:20:00, 2022-03-16 13:25:00)         NaN         NaN   
[2022-03-16 13:25:00, 2022-03-16 13:30:00)         NaN         NaN   
[2022-03-16 13:30:00, 2022-03-16 13:35:00)  289.109985  290.829987   
[2022-03-16 13:35:00, 2022-03-16 13:40:00)  290.209991  292.230011   
[2022-03-16 13:40:00, 2022-03-16 13:45:00)  291.730011  291.980011   

symbol                                                              \
                                                   low       close   
[2019-03-18, 2019-03-18)                    116.050003  117.570000   
[2019-03-19, 2019-03-19)                    116.989998  117.650002   
[2019-03-20, 2019-03-20)                    116.709999  117.519997   
[2019-03-21, 2019-03-21)                    117.089996  120.220001   
[2019-03-22, 2019-03-22)                    117.040001  117.050003   
...                                                ...         ...   
[2022-03-16 13:20:00, 2022-03-16 13:25:00)         NaN         NaN   
[2022-03-16 13:25:00, 2022-03-16 13:30:00)         NaN         NaN   
[2022-03-16 13:30:00, 2022-03-16 13:35:00)  289.029999  290.190002   
[2022-03-16 13:35:00, 2022-03-16 13:40:00)  289.940002  291.747711   
[2022-03-16 13:40:00, 2022-03-16 13:45:00)  290.549988  290.750000   

symbol                                                   AZN.L          \
                                                volume    open    high   
[2019-03-18, 2019-03-18)                    31207600.0  6290.0  6364.0   
[2019-03-19, 2019-03-19)                    37588700.0  6332.0  6405.0   
[2019-03-20, 2019-03-20)                    28113300.0  6373.0  6467.0   
[2019-03-21, 2019-03-21)                    29854400.0  6400.0  6540.0   
[2019-03-22, 2019-03-22)                    33624500.0  6468.0  6468.0   
...                                                ...     ...     ...   
[2022-03-16 13:20:00, 2022-03-16 13:25:00)         NaN  9456.0  9459.0   
[2022-03-16 13:25:00, 2022-03-16 13:30:00)         NaN  9454.0  9467.0   
[2022-03-16 13:30:00, 2022-03-16 13:35:00)   1794399.0  9462.0  9463.0   
[2022-03-16 13:35:00, 2022-03-16 13:40:00)    904231.0  9450.0  9463.0   
[2022-03-16 13:40:00, 2022-03-16 13:45:00)    620692.0  9460.0  9461.0   

symbol                                                                      \
                                               low        close     volume   
[2019-03-18, 2019-03-18)                    6290.0  6350.000000  1262164.0   
[2019-03-19, 2019-03-19)                    6332.0  6368.000000  1469458.0   
[2019-03-20, 2019-03-20)                    6373.0  6401.000000  1674037.0   
[2019-03-21, 2019-03-21)                    6400.0  6525.000000  1782299.0   
[2019-03-22, 2019-03-22)                    6335.0  6351.000000  1923870.0   
...                                            ...          ...        ...   
[2022-03-16 13:20:00, 2022-03-16 13:25:00)  9450.0  9455.000000     5093.0   
[2022-03-16 13:25:00, 2022-03-16 13:30:00)  9445.0  9463.302734    20563.0   
[2022-03-16 13:30:00, 2022-03-16 13:35:00)  9447.0  9448.000000    25119.0   
[2022-03-16 13:35:00, 2022-03-16 13:40:00)  9448.0  9459.000000    18785.0   
[2022-03-16 13:40:00, 2022-03-16 13:45:00)  9442.0  9452.000000    20088.0   

symbol                                     9988.HK                        
                                              open high low close volume  
[2019-03-18, 2019-03-18)                       NaN

Notice that the method gets composite tables whenever these can represent the period end with greater accuracy than a table with a regular interval could.

The indice will always be timezone aware, by default with timezone as the Prices instance's default timezone (`prices.tz_default`). This can be changed by passing either or both `tzin` and `tzout`, with the same consequences as for `get`.

In [25]:
tz = ZoneInfo("Australia/Perth")
minute_ = minute.astimezone(tz)
print(f"{minute_=}\n")  # for reference

rng = prices.price_range(end=minute, hours=5, tzout=tz)
rng

minute_=Timestamp('2022-03-16 21:45:00+0800', tz='Australia/Perth')



symbol                                            MSFT              \
                                                  open        high   
(2022-03-16 16:45:00, 2022-03-16 21:45:00]  289.109985  292.230011   

symbol                                                                     \
                                                   low   close     volume   
(2022-03-16 16:45:00, 2022-03-16 21:45:00]  289.029999  290.75  3319322.0   

symbol                                       AZN.L                            \
                                              open         high          low   
(2022-03-16 16:45:00, 2022-03-16 21:45:00]  9432.0  9481.019531  9366.302734   

symbol                                                        9988.HK       \
                                             close     volume    open high   
(2022-03-16 16:45:00, 2022-03-16 21:45:00]  9452.0  1267205.0     NaN  NaN   

symbol                                                       
                                           low close volume  
(2022-03-16 16:45:00, 2022-03-16 21:45:00] NaN   NaN    0.0

In [26]:
rng.index.left.tz == tz == rng.pt.tz

True

`lead_symbol` can be passed to define the calendar that the period should be evaluated against. As with `get`, this will also determine the timezone of the output (passing `tzout` will override this effect).

In [27]:
print(f"{minute=}\n")  # for reference

rng = prices.price_range(end=minute, hours=5, lead_symbol="MSFT")
rng

minute=Timestamp('2022-03-16 13:45:00+0000', tz='UTC')



symbol                                            MSFT              \
                                                  open        high   
(2022-03-15 11:15:00, 2022-03-16 09:45:00]  281.954987  292.230011   

symbol                                                                      \
                                                   low   close      volume   
(2022-03-15 11:15:00, 2022-03-16 09:45:00]  281.410004  290.75  21769747.0   

symbol                                       AZN.L                       \
                                              open         high     low   
(2022-03-15 11:15:00, 2022-03-16 09:45:00]  9326.0  9481.019531  9255.0   

symbol                                                           9988.HK  \
                                             close     volume       open   
(2022-03-15 11:15:00, 2022-03-16 09:45:00]  9452.0  2552609.0  74.599998   

symbol                                                                       \
                                                 high        low      close   
(2022-03-15 11:15:00, 2022-03-16 09:45:00]  93.900002  73.699997  90.699997   

symbol                                                   
                                                 volume  
(2022-03-15 11:15:00, 2022-03-16 09:45:00]  137009564.0

In [28]:
rng.pt.tz

<DstTzInfo 'America/New_York' LMT-1 day, 19:04:00 STD>

In [29]:
rng.pt.tz == prices.calendars["MSFT"].tz

True

`strict` can also be passed, with the sole effect of preventing an error from being raised if the period would otherwise start before the first day for which data is available.

In [30]:
start_oob = prices.limit_daily - pd.Timedelta(7, "D")
start_oob

Timestamp('1986-03-06 00:00:00')

In [ ]:
prices.price_range(start_oob)

```
---------------------------------------------------------------------------
StartTooEarlyError                        Traceback (most recent call last)
<ipython-input-40-58a73eaabb9d> in <module>
----> 1 prices.price_range(start_oob)

StartTooEarlyError: Prices unavailable as start (1986-03-06) is earlier than the earliest session for which price data is available. The earliest session for which prices are available is 1986-03-13.
```

In [32]:
prices.price_range(start_oob, strict=False)

symbol                                          MSFT                        \
                                                open        high       low   
(1986-03-13 02:00:00, 2022-05-13 14:26:00]  0.088542  349.670013  0.088542   

symbol                                                                \
                                                 close        volume   
(1986-03-13 02:00:00, 2022-05-13 14:26:00]  255.350006  5.347656e+11   

symbol                                           AZN.L                       \
                                                  open     high         low   
(1986-03-13 02:00:00, 2022-05-13 14:26:00]  600.322998  11000.0  580.580017   

symbol                                                            9988.HK  \
                                              close        volume    open   
(1986-03-13 02:00:00, 2022-05-13 14:26:00]  10250.0  2.641094e+10   187.0   

symbol                                                                   \
                                                  high   low      close   
(1986-03-13 02:00:00, 2022-05-13 14:26:00]  309.399994  71.0  82.199997   

symbol                                                    
                                                  volume  
(1986-03-13 02:00:00, 2022-05-13 14:26:00]  1.875208e+10

`stack` can be passed to stack the symbols to separate rows.

In [33]:
prices.price_range("2021", "2021-12-31", stack=True)

open         high  \
                                           symbol                              
(2021-01-04 01:30:00, 2021-12-31 21:00:00] 9988.HK   224.800003   270.000000   
                                           AZN.L    7425.000000  9523.000000   
                                           MSFT      222.529999   349.670013   

                                                            low        close  \
                                           symbol                              
(2021-01-04 01:30:00, 2021-12-31 21:00:00] 9988.HK   109.199997   118.900002   
                                           AZN.L    6499.799805  8678.000000   
                                           MSFT      211.940002   336.320007   

                                                          volume  
                                           symbol                 
(2021-01-04 01:30:00, 2021-12-31 21:00:00] 9988.HK  7.807695e+09  
                                           AZN.L    6.918552e+08  
                                           MSFT     6.555279e+09

And finally `include` and `exclude` can be passed with the same effect as for `get`.

In [34]:
prices.price_range("2021", months=3, include="9988.HK")

symbol                                         9988.HK                \
                                                  open   high    low   
(2021-01-04 01:30:00, 2021-04-01 21:00:00]  224.800003  270.0  214.0   

symbol                                                                
                                                 close        volume  
(2021-01-04 01:30:00, 2021-04-01 21:00:00]  225.600006  2.090326e+09

In [35]:
prices.price_range(days=5, exclude=["MSFT", "AZN.L"])

symbol                                        9988.HK                        \
                                                 open       high        low   
(2022-05-06 14:15:00, 2022-05-13 14:15:00]  83.599998  87.949997  79.800003   

symbol                                                              
                                                close       volume  
(2022-05-06 14:15:00, 2022-05-13 14:15:00]  82.199997  200227610.0